#Multivariate Causal Graph Fuzzy (MIMO)

In [ ]:
!pip install --quiet git+https://github.com/PatriciaLucas/AutoML.git
!pip3 install -U git+https://github.com/PYFTS/pyFTS
!git clone https://github.com/FutureLab-DCC/CGF-LLM.git

In [2]:
from AUTODCETS import datasets
import pandas as pd
import numpy as np
import sys
sys.path.append('CGF-LLM')
import MCGFuzzy, utils

In [3]:
name_dataset = 'SONDA'
target = 'glo_avg'
df = utils.get_dataset(name = name_dataset)

max_lags = 20
partitions = 30
database_path = 'MCGFuzzy.db'
epochs = 20
model_name = 'MCGFuzzy'

windows = utils.rolling_window(df, 10)
utils.execute("CREATE TABLE IF NOT EXISTS results(model TEXT, name_dataset TEXT, window INT, variable TEXT, predict FLOAT, \
               real FLOAT)", database_path)

[]

In [4]:
# Loop through the windows
for i, window in enumerate(windows):
    if i < 0:
      print('window already executed')
    else:
      # Exclude constant series.
      for variable in window.columns:
          if window[variable].max() == window[variable].min():
              window = window.drop(variable, axis=1)
              print(f"Variables {variable} were deleted because they are constant.")

      idx = int(0.8 * len(window))
      train_dataset, test_dataset = window.head(idx), window.tail(len(window) - idx)

      # MCGFuzzy
      ts_fuzzy, partitioners = MCGFuzzy.fuzzyfy(train_dataset, npart=partitions)
      model, graph = MCGFuzzy.fit(train_dataset, ts_fuzzy, target, max_lags)

      forecasts = MCGFuzzy.predict(test_dataset, model, graph, max_lags, partitioners)
      real = test_dataset[max_lags:]
      for row in range(forecasts.shape[0]):
          for v in forecasts.columns:
            utils.execute_insert("INSERT INTO results VALUES(?, ?, ?, ?, ?, ?)", (model_name, name_dataset, i, v, float(forecasts[v].iloc[row]), float(real[v].iloc[row])), database_path)

      print(f'Save window: {i}')

Save window: 0
Save window: 1
Save window: 2


## How to view the results

In [5]:
results = utils.get_metrics_multivariate(database_path)
results

,model,name_dataset,variable,nrmse
0,MCGFuzzy,SONDA,diff_avg,0.166209
1,MCGFuzzy,SONDA,dir_avg,0.329318
2,MCGFuzzy,SONDA,glo_avg,0.150252
3,MCGFuzzy,SONDA,humid,0.292352
4,MCGFuzzy,SONDA,lux_avg,0.288685
5,MCGFuzzy,SONDA,lw_avg,1.104424
6,MCGFuzzy,SONDA,par_avg,0.283926
7,MCGFuzzy,SONDA,press,0.709578
8,MCGFuzzy,SONDA,rain,0.519642
9,MCGFuzzy,SONDA,tp_sfc,0.530664
